In [1]:
import numpy as np
import json, toml  
import json
import matplotlib.pyplot as plt
from shapely.geometry import Point
from shapely.geometry.polygon import Polygon

In [2]:
# def get_focmec(elat,elon, edep=None, Mw = None, regime= 'crust', pdf=True, focmec=None)
# strike360 = strike%360
# focmec_model = json.load(f)

In [ ]:
def get_ntdomain(xlon, xlat, config):
    # for a given lon, lat, return the domain
    file = config['modelfolder'] + config['ntdomains']['file']
    with open(file) as f:
        ntdomains = json.load(f)
        
    point = Point(xlon, xlat)
    domain = None

    for dn in ntdomains.keys():
        lons = ntdomains[dn]['lon']
        lats = ntdomains[dn]['lat']
        dpoints = []
        for i in range(len(lons)):
            dpoints.append((lons[i], lats[i]))
        polygon = Polygon(dpoints)
        if polygon.contains(point):
            domain = dn
            break;
            
    return domain

In [25]:
#---------------------------------------------------------------------------
def get_sdrp_crust(fmodel, mag, prefmod = 'mean_all'):
    #  
    # prefmod can one of following:
    # 'case1', 'case2', 'mean_case1', 'mean_case2', 'all', 'mean_all'
    # 
    
    if mag is None:
         raise Exception("Mag cannot be None")
    
    def isequal_fm(fm1, fm2):
        #fm = sdrp 
        for x1, x2 in zip(fm1, fm2):
            if x1!=x2: return False
        return True
   
    if prefmod in ['case1','case2']:
        tag = '>15' if mag<=7.5 else '>45'
        if tag not in fmodel[prefmod].keys():
            tag = '>15'      
        #
        strike = fmodel[prefmod][tag]['strikeAn']
        dip = fmodel[prefmod][tag]['dipAn']
        rake = fmodel[prefmod][tag]['rakeAn']
        prob = fmodel[prefmod][tag]['prob']
        sdrp = []
        for s,d,r,p in zip(strike, dip, rake, prob):
            sdrp.append([s,d,r,p])
            
        return sdrp
    
    elif prefmod in ['mean_case1','mean_case2']:
        # flength_tag = ['>15', '>15', '>45']
        pmod = prefmod[-5:]  # i.e. "case1" if "mean_case1"
        tag = '>15' if mag<=7.5 else '>45'
        if tag not in fmodel[pmod].keys():
            tag = '>15'  
        #
        strike = fmodel[pmod][tag]['strikeAn']
        dip = fmodel[pmod][tag]['dipAn']
        rake = fmodel[pmod][tag]['rakeAn']
        prob = fmodel[pmod][tag]['prob']
        mind = prob.index(max(prob))
        sdrp = [[strike[mind], dip[mind], rake[mind], 1.0],]
        return sdrp
    
    elif prefmod in ['all','mean_all']:
        if prefmod=='all':
            sdrp1 = get_sdrp_crust(fmodel, mag, prefmod = 'case1')
            sdrp2 = get_sdrp_crust(fmodel, mag, prefmod = 'case2')
        else:
            sdrp1 = get_sdrp_crust(fmodel, mag, prefmod = 'mean_case1')
            sdrp2 = get_sdrp_crust(fmodel, mag, prefmod = 'mean_case2')
        
        if isequal_fm(sdrp1, sdrp2):
            sdrp = sdrp1
        else:
            sdrp = []
            for x in sdrp2:
                sdrp1.append(x)
            for x in sdrp1:
                sdrp.append([x[0],x[1], x[2], x[3]*0.5]) 
        
        return sdrp
    
    raise Exception("prefmod  should be either 'case1', \
                'case2','mean_case1','mean_case2','all', or 'mean_all'")

In [26]:
def get_sdrp(elon,elat, edep=None, Mw = None, regime= 'crust', \
               preferred_model = 'mean_all', modelfolder = '../',
               config_file = 'models/nz-focmec_v1.ini'):
    # load config
    config = toml.load(modelfolder+config_file)
    config['modelfolder'] = modelfolder
    regime = regime.lower()
    # if-else is faster than select
    if regime == 'crust':
        # locate ntdomain
        dom = get_ntdomain(elon, elat, config)
        with open(modelfolder+config[regime]['file']) as f:
            focmod = json.load(f)
        if dom is None:
            raise Exception("Lon, Lat is outside the domain.")
        else:
            sdrp = get_sdrp_crust(focmod[dom], Mw, prefmod = preferred_model)            
    elif regime=='slab':
        # identify subduction zone 
        
        with open(modelfolder+file) as f:
            mod = json.load(f)
        pass
    elif regime=='interface':
        # identify subduction zone 
        pass
    else:    
        raise Exception("regime should be either 'crust','slab' or 'interface'")
        
    

        

In [27]:
get_focmec(170, -45, edep=None, Mw = 6, regime= 'crust', \
               preferred_model = 'mean_all', pdf=True, focmec=None,)

[[220, 45, 115, 1.0]]


In [24]:
pdf=True, focmec=None,
# slab
focfile = /home/kirant/thinglap/GitHubRepo/nz-focmec/models/slab/nz-slab-focmec.json

focmec_model = json.load(f)

# Focal-mechanism model or NodalPlaneDistribution
# faulting_model has items that can be either of three:
#    (1) dictionary - fm, pref
#    (2) interpolator of strike and dip, where rake = 90
#    (3) dictionary with keys - strike dip rake probability
# It may also make sense to organize the fochal mechanism according to regimes


[slab]
# the json format is:
# szone: {dep: [[s1, d1, r1, p1], [s2, d2, r2, p2], .... ]}
# See Model-development/9RuptureGeometry/workout-sdrp-slabrupgeom
file = 'components/faulting_model/slab-faulting2.json'
magbins = [[5.0, 6.5], [6.6, 8.15]]


SyntaxError: invalid syntax (3954234837.py, line 2)